In [2]:
from random import randrange
from typing import List

from networkx import Graph

graphs: List[Graph] = []

graph_sizes = [4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20]
num_graph_per_size = 4

for size in graph_sizes:
    for i in range(num_graph_per_size):
        graph = Graph()
        num_edges = randrange(2, size * (
                    size - 2))  # rand in range of max number of possible edges
        edges = []

        for _ in range(num_edges):  # add random number of edges

            u = randrange(size - 1)
            v = randrange(size - 1)
            w = randrange(1, 4)

            if (u, v) not in [(u1, v1) for (u1, v1, _) in edges]:
                edges.append((u, v, w))

        for node in range(size):  # add an edge for any nodes not connected in the graph
            if node not in [u for (u, _, _) in edges] and node not in [v for (_, v, _)
                                                                       in edges]:
                u = node
                v = randrange(size - 1)
                w = randrange(1, 4)

                edges.append((u, v, w))

        graph.add_weighted_edges_from(edges)
        graphs.append(graph)

len(graphs)

48

In [14]:
import json
from pathlib import Path

from networkx import to_dict_of_dicts

with open(Path("graphs.json"), "w") as file:
    json.dump(
        {key: to_dict_of_dicts(value) for key, value in enumerate(graphs)}, file
    )

In [36]:
from copy import deepcopy
import json
from pathlib import Path
from bachelorthesis.graph_optimization import GraphOptimization

from networkx import from_dict_of_dicts

from bachelorthesis.solvers.qb_solv import solve as solve_qbs
from bachelorthesis.solvers.repeated_simulated_annealing import solve as solve_rsa
from bachelorthesis.solvers.simulated_annealing import solve as solve_sa
from bachelorthesis.solvers.tabu_search import solve as solve_tbs

with open(Path("graphs.json"), "r") as graphs_file:
    saved_graphs = json.load(graphs_file)

with open(Path("graph_optimizations.json"), "r") as g_opts_file:
    optimization_parameters = json.load(g_opts_file)

for graph_id, graph in saved_graphs.items():
    # dict_of_dicts turns int keys into strings, this converts them back to ints
    graph_data = {int(key): {int(key2): value for key2, value in graph.get(key).items()} for key, _ in graph.items()}
    graph = from_dict_of_dicts(graph_data)

    # create graph optimization
    g_opt = GraphOptimization(graph=graph)
    start_node = 0
    terminal_node = graph.order()

    results = []

    for params_id, params in optimization_parameters.items():
        # allow positions in the json to take on a value of "n" for faster problem creation
        params_copy = deepcopy(params)
        pos = params_copy.pop("positions")
        if pos == "n":
            pos = graph.order()

        # generate qubo
        qubo = g_opt.generate_qubo(
            positions=pos,
            start_node=start_node,
            terminal_node=terminal_node,
            **params_copy
        )

        # solve qubo and store result with all the relevant solvers

        solution_qbs = solve_qbs(qubo)
        solution_rsa = solve_rsa(qubo, num_repeats=10)
        solution_sa = solve_sa(qubo)
        solution_tbs = solve_tbs(qubo)

        result_entry = {
            "graph_id": graph_id,
            "params_id": params_id,
            "qubo": qubo.tolist(),
            "solution_qbs_sample": solution_qbs[0][0].tolist(),
            "solution_qbs_energy": solution_qbs[1].tolist()[0],
            "solution_rsa_sample": solution_rsa[0][0].tolist(),
            "solution_rsa_energy": solution_rsa[1].tolist()[0],
            "solution_sa_sample": solution_sa[0][0].tolist(),
            "solution_sa_energy": solution_sa[1].tolist()[0],
            "solution_tbs_sample": solution_tbs[0][0].tolist(),
            "solution_tbs_energy": solution_tbs[1].tolist()[0]
        }
        results.append(result_entry)

    # store results per graph because i/o is relatively expensive
    with open(Path("results.json"), "r") as results_file_in:
        current_results = json.load(results_file_in)
        current_results.extend(results)
    with open(Path("results.json"), "w") as results_file_out:
        json.dump(current_results, results_file_out)


[[ -5.  10.   0.   0.]
 [  0.  -5.  10.   0.]
 [  0.   0. -10.  10.]
 [  0.   0.   0.  -5.]]
(array([[1, 0, 1, 0]], dtype=int8), array([-15.]))
(array([[1, 0, 1, 0]], dtype=int8), array([-15.]))
(array([[1, 0, 1, 0]], dtype=int8), array([-15.]))
(array([[1, 0, 1, 0]], dtype=int8), array([-15.]))
[[-10.  10.  10.   0.]
 [  0.  -5.   0.   0.]
 [  0.   0. -10.  10.]
 [  0.   0.   0.  -5.]]
(array([[0, 1, 1, 0],
       [1, 0, 0, 1]], dtype=int8), array([-15., -15.]))
(array([[0, 1, 1, 0]], dtype=int8), array([-15.]))
(array([[1, 0, 0, 1]], dtype=int8), array([-15.]))
(array([[1, 0, 0, 1]], dtype=int8), array([-15.]))
[[ -5.  10.   0.   0.]
 [  0. -15.  10.  10.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.  -5.]]
(array([[0, 1, 0, 0]], dtype=int8), array([-15.]))
(array([[0, 1, 0, 0]], dtype=int8), array([-15.]))
(array([[0, 1, 0, 0]], dtype=int8), array([-15.]))
(array([[0, 1, 0, 0]], dtype=int8), array([-15.]))
[[-15.  10.  10.  10.]
 [  0.  -5.   0.   0.]
 [  0.   0.  -5.   0.]
 [  0.   0.  

In [ ]:
# solvers to use?
# nico used SA, RRSA, SAGA, TABU
# parameterization?